In [1]:
#@title DPO 방식으로 모델 재학습 (A100 최적화 및 CUDA 오류 해결 최종판 - 2025 업데이트)

# --- 1. ★ 최종 라이브러리 설치 ---
# 2025년 기준 최신 버전 + bitsandbytes 호환 설치 (CUDA 12.6 지원)
!pip install -q -U transformers peft accelerate trl datasets huggingface_hub bitsandbytes fastapi "uvicorn[standard]" pyngrok
!pip install -q "pandas==2.2.2"

In [2]:
# --- 2. 파일 업로드 및 로그인 ---
from google.colab import files
from huggingface_hub import login
import os

# ❗ 최종 DPO 모델 zip 파일과 styles.csv를 업로드해주세요.
files_to_upload = ['gemma-fashion-dpo-final-v3.zip', 'styles.csv']
for filename in files_to_upload:
    if not os.path.exists(filename):
        print(f"'{filename}'을 업로드해주세요.")
        files.upload()
    else:
        print(f"'{filename}'이(가) 이미 존재합니다.")

# 모델 폴더 압축 해제
print("\n모델 폴더 압축을 해제합니다...")
!unzip -q -o gemma-fashion-dpo-final-v3.zip
adapter_path = "/content/content/gemma-fashion-dpo-final-v3"
print(f"✅ 모델 경로 확인: {adapter_path}")

# Hugging Face 로그인
print("\n--- 🔐 Hugging Face 로그인이 필요합니다 ---")
login()
print("✅ 로그인 성공!")

'gemma-fashion-dpo-final-v3.zip'이(가) 이미 존재합니다.
'styles.csv'이(가) 이미 존재합니다.

모델 폴더 압축을 해제합니다...
✅ 모델 경로 확인: /content/content/gemma-fashion-dpo-final-v3

--- 🔐 Hugging Face 로그인이 필요합니다 ---


✅ 로그인 성공!


In [3]:
#@title 3. API 서버 코드 작성 (serve_final.py) - 최종 하이브리드 스코어링 버전

%%writefile serve_final.py

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
import pandas as pd
from itertools import product
import time
from fastapi import FastAPI
from pydantic import BaseModel
from typing import List, Dict, Any
import uvicorn

# --- 1. 모델 로딩 ---
print("✅ 최종 DPO 모델 로딩을 시작합니다...")
device = "cuda" if torch.cuda.is_available() else "cpu"
adapter_path = "/content/content/gemma-fashion-dpo-final-v3"
quantization_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16)
base_model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it", quantization_config=quantization_config, device_map="auto")
model = PeftModel.from_pretrained(base_model, adapter_path)
model.eval()
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
tokenizer.pad_token = tokenizer.eos_token
positive_token_id = tokenizer.encode("긍정적", add_special_tokens=False)[0]
negative_token_id = tokenizer.encode("부정적", add_special_tokens=False)[0]
print("✅ 모델 및 토크나이저 로딩 완료!")

# --- 2. 데이터 및 FastAPI 앱 설정 ---
styles_df = pd.read_csv("styles.csv", on_bad_lines='skip', dtype={'id': int}).set_index('id')
styles_info = styles_df.to_dict('index')
print("✅ styles.csv 로드 완료!")
app = FastAPI()

def get_item_description(item_id):
    if item_id not in styles_info: return ""
    info = styles_info[item_id]; season = info.get('season', 'All')
    return f"{info.get('baseColour', '')} {info.get('articleType', '')} ({season})".strip()

# --- ★ NEW: 규칙 기반 '상식 점수' 계산 함수 ---
def calculate_rule_score(outfit_ids: List[int], context: Dict[str, Any]) -> float:
    """상황과 옷의 용도(usage)가 얼마나 잘 맞는지 점수를 매기는 함수"""
    score = 0.0
    items_info = [styles_info.get(id) for id in outfit_ids if id in styles_info]

    # 이벤트에 따른 기대 용도 설정
    event = context['event']
    expected_usages = []
    if event in ["Business Meeting", "Job Interview", "Formal Dinner", "Wedding Ceremony"]: expected_usages = ['Formal', 'Smart Casual']
    elif event in ["Gym Workout", "Hiking", "Sports"]: expected_usages = ['Sports', 'Active']
    else: expected_usages = ['Casual', 'Smart Casual']

    # 각 아이템의 용도가 기대 용도에 맞는지 확인
    for item in items_info:
        if item and item.get('usage') in expected_usages:
            score += 10 # 잘 맞으면 10점
        else:
            score -= 5 # 안 맞으면 감점

    return score

class OutfitRequest(BaseModel):
    closet: List[int]; event: str; temperature: float; condition: str; gender: str

@app.post("/recommend_outfit")
def recommend_outfit(req: OutfitRequest):
    start_time = time.time()
    # (이전과 동일한 필터링 및 분류, 조합 생성 로직)
    # ... (생략) ...
    temp = req.temperature
    if temp <= 10: suitable_seasons = ['Winter', 'Fall']
    elif temp <= 18: suitable_seasons = ['Fall', 'Winter', 'Spring', 'All Season']
    else: suitable_seasons = ['Summer', 'Spring', 'All Season']
    closet_items = { 'Topwear': [], 'Bottomwear': [], 'Outerwear': [], 'Footwear': [], 'Full Body': [] }
    outerwear_types = ['Jackets', 'Blazers', 'Waistcoat', 'Coat', 'Shrug', 'Cardigan', 'Nehru Jackets', 'Rain Jacket', 'Sweaters']
    full_body_types = ['Dresses', 'Jumpsuit']
    for item_id in req.closet:
        if item_id in styles_info:
            info = styles_info[item_id]
            if info.get('season') in suitable_seasons or info.get('season') == 'All Season' or info.get('usage') == 'Sports':
                sub_category = info.get('subCategory', ''); article_type = info.get('articleType', ''); master_category = info.get('masterCategory', '')
                if article_type in full_body_types: closet_items['Full Body'].append(item_id)
                elif article_type in outerwear_types: closet_items['Outerwear'].append(item_id)
                elif sub_category == 'Topwear': closet_items['Topwear'].append(item_id)
                elif sub_category == 'Bottomwear': closet_items['Bottomwear'].append(item_id)
                elif master_category == 'Footwear': closet_items['Footwear'].append(item_id)
    base_combinations = list(product(closet_items.get('Topwear', []), closet_items.get('Bottomwear', []), closet_items.get('Footwear', [])))
    outer_combinations = list(product(closet_items.get('Topwear', []), closet_items.get('Bottomwear', []), closet_items.get('Outerwear', []), closet_items.get('Footwear', [])))
    full_body_combinations = list(product(closet_items.get('Full Body', []), closet_items.get('Footwear', [])))
    all_combinations = [c for c in base_combinations + outer_combinations + full_body_combinations if c]

    if not all_combinations: return {"error": "현재 날씨와 상황에 맞는 유효한 조합을 옷장에서 찾을 수 없습니다."}

    prompts = [ f"상황: {req.gender}, {req.temperature}°C, {req.condition}, {req.event}\n옷: {', '.join([get_item_description(id) for id in combo])}\n이 조합은 적절한가요?\n결과:" for combo in all_combinations ]

    # --- AI 점수 계산 ---
    inputs = tokenizer(prompts, return_tensors="pt", padding=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    sequence_lengths = inputs['attention_mask'].sum(dim=1) - 1
    last_token_logits = outputs.logits[torch.arange(len(all_combinations)), sequence_lengths]
    logits_for_scoring = last_token_logits[:, [negative_token_id, positive_token_id]]
    probs = torch.softmax(logits_for_scoring, dim=-1)
    ai_scores = probs[:, 1].tolist()

    # --- ★ NEW: 하이브리드 스코어링 ---
    final_scored_combinations = []
    for i, combo in enumerate(all_combinations):
        combo_ids = [int(id_val) for id_val in combo if id_val is not None]
        rule_score = calculate_rule_score(combo_ids, req.dict())
        ai_score = ai_scores[i]

        # 상식 점수 70%, AI 점수 30%를 합산하여 최종 점수 계산
        final_score = (rule_score * 0.7) + (ai_score * 0.3)
        final_scored_combinations.append((combo, final_score))

    # 최종 점수가 가장 높은 순으로 정렬
    sorted_combinations = sorted(final_scored_combinations, key=lambda x: x[1], reverse=True)

    if not sorted_combinations:
        return {"error": "추천할 만한 조합을 찾지 못했습니다."}

    best_combo, best_score = sorted_combinations[0]
    best_combo_ids = [int(id_val) for id_val in best_combo if id_val is not None]
    outfit_str = ", ".join([get_item_description(id) for id in best_combo_ids])
    best_outfit_info = {"description": outfit_str, "ids": best_combo_ids}

    explanation = f"긍정적: {req.temperature}°C의 {req.condition} 날씨에 진행되는 {req.event}에 가장 잘 어울리는 스타일입니다."
    end_time = time.time()
    return {"best_combination": best_outfit_info, "explanation": explanation, "best_score": best_score, "processing_time": end_time - start_time}

Overwriting serve_final.py


In [4]:
#@title 4. 서버 실행 및 테스트 (최종 수정)

# --- 1. [진단] 현재 할당된 GPU 확인 ---
# 이 명령어를 실행했을 때 'Tesla T4' 또는 'A100' 등이 보이면 GPU가 정상적으로 할당된 것입니다.
!nvidia-smi

# --- 2. 서버 실행 및 ngrok 터널 생성 ---
import threading
import time
import requests
from pyngrok import ngrok
import uvicorn  # ❗ NameError 해결을 위해 uvicorn을 import 합니다.

def run_app():
    # serve_final.py를 uvicorn으로 실행 (reload=True로 개발 중 핫 리로딩 추가 가능)
    uvicorn.run("serve_final:app", host="0.0.0.0", port=8000, log_level="info", reload=False)  # reload=False로 안정성 높임

thread = threading.Thread(target=run_app)
thread.start()
print("\n✅ FastAPI 서버가 백그라운드에서 실행을 시작했습니다...")

# --- 서버가 켜질 때까지 대기 (Health Check) ---
print("⏳ 서버가 모델을 로딩하고 준비될 때까지 대기 중입니다... (최대 10분)")
print("   (GPU 사용량 확인: 이 셀 실행 중 새 코드 셀을 추가하여 !nvidia-smi 를 실행해보세요)")
start_time = time.time()
server_ready = False
while time.time() - start_time < 600:
    try:
        response = requests.get("http://localhost:8000/", timeout=5)
        if response.status_code == 404:  # FastAPI 기본 응답 (루트가 없으면 404)
            print("\n✅ 서버가 응답하기 시작했습니다! 모델 로딩 완료.")
            server_ready = True
            break
    except requests.exceptions.ConnectionError:
        time.sleep(5)
        print("... 대기 중 (모델 로딩 중일 수 있음)")

if not server_ready:
    print("\n❌ 10분이 지나도 서버가 준비되지 않았습니다. 런타임을 재시작하고 serve_final.py를 확인해주세요.")
else:
    # --- ngrok으로 외부 접속 터널 생성 ---
    NGROK_TOKEN = "31HHkLEWGt90qDRoAWd6BqiGL9K_5fJsXN7LgijbtcAVwkwAS"  # 🚨 당신의 토큰
    ngrok.set_auth_token(NGROK_TOKEN)
    ngrok.kill()
    public_url = ngrok.connect(8000)
    print(f"\n🎉 서버가 성공적으로 생성되었습니다. API 주소: {public_url}")
    print("이제 이 주소를 test_api.py에 넣고 로컬에서 테스트를 실행하세요.")

# thread.join()은 Colab에서 무한 대기할 수 있으니, 필요 시 주석 처리 (백그라운드 실행 유지)
# thread.join()

Wed Aug 27 11:26:10 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   65C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

... 대기 중 (모델 로딩 중일 수 있음)
... 대기 중 (모델 로딩 중일 수 있음)
✅ 모델 및 토크나이저 로딩 완료!


INFO:     Started server process [59485]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


✅ styles.csv 로드 완료!
... 대기 중 (모델 로딩 중일 수 있음)
INFO:     127.0.0.1:49446 - "GET / HTTP/1.1" 404 Not Found

✅ 서버가 응답하기 시작했습니다! 모델 로딩 완료.

🎉 서버가 성공적으로 생성되었습니다. API 주소: NgrokTunnel: "https://a3445d597d22.ngrok-free.app" -> "http://localhost:8000"
이제 이 주소를 test_api.py에 넣고 로컬에서 테스트를 실행하세요.


In [6]:
!ls -lR /content/content/gemma-fashion-dpo-final-v3

/content/content/gemma-fashion-dpo-final-v3:
total 45000
-rw-r--r-- 1 root root      880 Aug 27 07:03 adapter_config.json
-rw-r--r-- 1 root root  7391688 Aug 27 07:03 adapter_model.safetensors
-rw-r--r-- 1 root root      591 Aug 27 07:03 chat_template.jinja
drwxr-xr-x 2 root root     4096 Aug 27 07:46 checkpoint-100
drwxr-xr-x 2 root root     4096 Aug 27 07:46 checkpoint-200
drwxr-xr-x 2 root root     4096 Aug 27 07:46 checkpoint-300
drwxr-xr-x 2 root root     4096 Aug 27 07:46 checkpoint-400
drwxr-xr-x 2 root root     4096 Aug 27 07:46 checkpoint-482
-rw-r--r-- 1 root root     2453 Aug 27 07:03 README.md
-rw-r--r-- 1 root root      522 Aug 27 07:03 special_tokens_map.json
-rw-r--r-- 1 root root    40021 Aug 27 07:03 tokenizer_config.json
-rw-r--r-- 1 root root 34356041 Aug 27 07:03 tokenizer.json
-rw-r--r-- 1 root root  4241003 Aug 27 07:03 tokenizer.model
-rw-r--r-- 1 root root     6737 Aug 27 07:03 training_args.bin

/content/content/gemma-fashion-dpo-final-v3/checkpoint-100:
total 